# Optimizing QAOA

by Pantita Palittapongarnpim

Now that we have the classes for QAOA circuits, we need to optimize the circuits.

### Package Versions

QuTiP: 4.5.2

Python: 3.8.6

### Loading Packages

In [58]:
import numpy as np
import qutip as qt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

### QAOA Classes

In [2]:
class Graph:
    def __init__(self, total_qubit, J):
        self.total_qubit=total_qubit #integer
        self.J=J #matrix: undirected (symmetric)
        
    def Ham(self):
        Ham_out=0;
        for i in range(self.total_qubit):
            for j in range(i,self.total_qubit):
                opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
                Ham_out=Ham_out+J[i][j]*opr
        return(Ham_out)

In [86]:
class QAOA:
    def __init__(self, gamma, beta, p, graph):
        ## what does graph has to contain:
        # total_qubit -- integer
        # J -- double matrix        
        self.gamma=gamma
        self.beta=beta
        self.p=p
        self.J=graph.J
        self.total_qubit=graph.total_qubit
        
    def Uz(self,i,j,l):
        opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
        Hvar=-1j*self.gamma[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def Ux(self,i,l):
        opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)])
        Hvar=-1j*self.beta[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def circuit(self):
        Had=qt.tensor([qt.qip.operations.snot() for k in range(self.total_qubit)])
        qaoa_circ=Had
        for l in range(self.p):
            #operate the Uz gates
            for i in range(self.total_qubit):
                for j in range(i,self.total_qubit):
                    if i!= j:
                        qaoa_circ=self.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(self.total_qubit):
                qaoa_circ=self.Ux(i,l)*qaoa_circ
        return(qaoa_circ)
    
    def state(self):
        gnd=qt.basis(2,0)
        state=qt.tensor([gnd for i in range(self.total_qubit)])
        state=self.circuit()*state
        return(state)
 

### Objective functions

Now that we have the simulation of the QAOA circuit, we need to optimize the parameters, namely gamma and beta array.

But before that, we need to pick the objective function we are going to use.

__Average Energy__

This is the traditional objective function for QAOA

__Gibbs__

This one is proposed by Li et al.

__Average Gibbs__

Actually doesn't have a name, proposed by P'Thip.

In [98]:
#so this is our fun in scipy.optimize.minimize
# the form has to be fun(x, *arg)
# x here has to be gamma and beta
# arg* has to be graph and p

def AveEnergy(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    return((output.dag()*graph.Ham()*output).tr())

def Gibbs(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    f=-np.log((output.dag()*(-eta*graph.Ham()).expm()*output).tr())
    return(f)

def AveGibbs(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    f=-np.log((output.dag()*graph.Ham()*(-eta*graph.Ham()).expm()*output).tr())
    return(f)

### Testing out the objective functions

In [88]:
total_qubit=3 #number of qubit
p=1 #number of layers

#declaring QAOA parameters
gamma=np.random.rand(p)
beta=np.random.rand(p)

#defining the problem
J=np.random.rand(total_qubit,total_qubit) #creating the couping parameters,still directional in this instance)
for i in range(total_qubit):
    for j in range(i,total_qubit):
        temp=(J[i][j]+J[j][i])/2.0
        J[i][j]=temp
        J[j][i]=temp
#Now J is undirectional

In [91]:
graph = Graph(total_qubit,J)

x0=np.append(gamma,beta)
arg=(p,graph)

In [79]:
#Average Energy#
print(x0,AveEnergy(x0,*arg))

#now let's try minimize

res=minimize(AveEnergy,x0,arg,method='Nelder-Mead')

print(res.x,res.fun)

bounds=[(-2,2),(-2,2)]
res=differential_evolution(AveEnergy,bounds,args=arg)
print(res.x,res.fun)


[0.14022272 0.22302041] 0.12265013456946702
[-0.51961382  0.71118769] -0.6546572269149862
[-0.51963065  0.71118066] -0.6546572275903807


In [107]:
#Gibbs energy
eta=0.3 #how do we tune this again?
print(x0,Gibbs(x0,*arg))

res=minimize(Gibbs,x0,arg,method='Nelder-Mead')

print(res.x,res.fun)

bounds=[(-3,3),(-3,3)]
res=differential_evolution(Gibbs,bounds,args=arg)
print(res.x,res.fun)

[0.07284947 0.69721935] -0.08467304075945914
[2.30192118 1.85705565] -0.15513841217953755
[2.30187512 1.85706631] -0.15513841220770794


In [106]:
#Average Gibbs
eta=0.5
print(x0,AveGibbs(x0,*arg))

#res=minimize(AveGibbs,x0,arg,method='Nelder-Mead')

#print(res.x,res.fun)

#bounds=[(-2,2),(-2,2)]
#res=differential_evolution(AveGibbs,bounds,args=arg)
#print(res.x,res.fun)

[0.07284947 0.69721935] nan


<ipython-input-98-5500b78f2cb9>:41: RuntimeWarning: invalid value encountered in log
  f=-np.log((output.dag()*graph.Ham()*(-eta*graph.Ham()).expm()*output).tr())


Observations:

* As expected, the Gibbs energy depend on eta, but we don't have a good idea what value to use.
* Average Gibbs yields nan. Not sure what happens there yet.

### Nelder-Mead Simplex Algorithm

In [28]:
#Initializing simplex
#I am assuming Cartesian in matrix representation
def simplex_init(n,p,scale):
    #n is the number of simplex
    #p0 is the initial point
    
    x=scale*np.random.rand(n,2*p)
    return(x)

In [53]:
n=5
scale=5
x=simplex_init(n,2*p,scale)
f=np.zeros(n)
for i in range(n):
    f[i]=AveEnergy(x[i],*arg)
    print(x[i],f[i])
    


[0.92317108 1.79490746 4.38256974 2.96171123] -0.19087035110822373
[4.44681982 4.37682233 2.61441466 4.05387   ] 0.2743614387770656
[2.88551502 1.933726   0.45733092 1.90168599] 0.40925075963260726
[4.06462033 4.67548563 1.17780632 0.67966867] -0.0016225551778136782
[2.59340427 2.56909118 4.10859877 0.71299045] 0.18726774271259786
